In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import DataLoader 

device =torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device :",device)

transform=transforms.ToTensor()

train_data=MNIST(root="data", train=True, download=True, transform=transform)
test_data=MNIST(root="data", train=False, download=True, transform=transform)

train_loader=DataLoader(train_data, batch_size=64, shuffle=True)
test_loader=DataLoader(test_data, batch_size=64, shuffle=False)

class SimpleCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.features=nn.Sequential(
            nn.Conv2d(1, 8, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2),
            nn.Conv2d(8, 16, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2)
        )
        self.classifier=nn.Sequential(
            nn.Linear(16*7*7, 128),
            nn.ReLU(),
            nn.Linear(128, 10)
        )
    def forward(self,x):
        x=self.features(x)
        x=x.view(x.size(0), -1)  
        x=self.classifier(x)
        return x
    
model=SimpleCNN().to(device)
loss_fn=nn.CrossEntropyLoss()
optimizer=optim.Adam(model.parameters(), lr=0.001)
epochs=3
for epoch in range(epochs):
    model.train()
    total_loss=0
    for images, labels in train_loader:
        images, labels = images.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs= model(images)
        loss = loss_fn(outputs, labels)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch [{epoch+1}/{epochs}], Loss: {total_loss:.4f}")
model.eval()
correct, total = 0, 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        prediction =outputs.argmax(dim=1)
        total += labels.size(0)
        correct += (prediction == labels).sum().item()

print(f"Accuracy: {100 * correct / total:.2f}%")
torch.save(model.state_dict(), "simple_cnn.pth")
print("Model saved as simple_cnn.pth")
 

c:\Users\AADITYA PORWAL\.conda\envs\GPU\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Using device : cuda
Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 9912422/9912422 [00:10<00:00, 909514.10it/s] 


Extracting data\MNIST\raw\train-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 28881/28881 [00:00<00:00, 92284.79it/s]


Extracting data\MNIST\raw\train-labels-idx1-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 1648877/1648877 [00:03<00:00, 535493.28it/s]


Extracting data\MNIST\raw\t10k-images-idx3-ubyte.gz to data\MNIST\raw

Failed to download (trying next):
HTTP Error 404: Not Found



100%|██████████| 4542/4542 [00:00<00:00, 2270082.07it/s]


Extracting data\MNIST\raw\t10k-labels-idx1-ubyte.gz to data\MNIST\raw

Epoch [1/3], Loss: 289.1028
Epoch [2/3], Loss: 74.6449
Epoch [3/3], Loss: 52.5016
Accuracy: 98.63%
Model saved as simple_cnn.pth
